## 1. Naive Bayes classifcation for mini-newsgroups datasets

In [202]:
#这是使用传统的机器学习模型，而不是神经网络。
#coding的难点在于特征矩阵的构建，没有用到pytorch或者transformer框架

In [1]:
import os
import spacy
import string
import numpy as np
from spacy.lang.en import stop_words
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [2]:
# unzip the datasets.zip file, you will find there are some datasets (20_newsgroups, acllmdb, mini_newsgroups)
base_path = '/home/leoyang/Desktop/data/data-lec3/datasets/mini_newsgroups'
X,Y=[],[]
for category in os.listdir(base_path):
    if category.startswith('.'): 
        continue
    for document in os.listdir(os.path.join(base_path, category)):
        if document.startswith('.'): 
            continue
        with open(os.path.join(base_path, category, document), "r", errors='ignore') as f:
            X.append((document, f.read()))
            Y.append(category)

FileNotFoundError: [Errno 2] No such file or directory: '/home/leoyang/Desktop/data/data-lec3/datasets/mini_newsgroups'

In [190]:
print(X[2])
print()
print(Y[100])



('54659', 'Xref: cantaloupe.srv.cs.cmu.edu talk.politics.guns:54659 soc.culture.usa:13862 alt.rush-limbaugh:23770\nPath: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!wupost!uunet!pipex!uknet!keele!news\nFrom: phd85@seq1.keele.ac.uk (D.H. Holden)\nNewsgroups: talk.politics.guns,soc.culture.usa,alt.rush-limbaugh\nSubject: Re: ATF BURNS DIVIDIAN RANCH! NO SURVIVORS!!!\nDate: 21 Apr 1993 12:25:41 GMT\nLines: 9\nDistribution: world\nMessage-ID: <1r3eg6$qdi@gabriel.keele.ac.uk>\nReferences: <1qvjh9INNh4l@hp-col.col.hp.com>\nNNTP-Posting-Host: seq1.cc.keele.ac.uk\n\nFrom article <1qvjh9INNh4l@hp-col.col.hp.com>, by dduff@col.hp.com (Dave Duff):\n> NUT CASE PANICS!!!!JUMPS THE GUN ON THE NET BEFORE GETTING FACTS STRAIGHT!!!!\n\n   Brilliant I like it!\n--\nDave Holden Phys. Dept. |  Email:                          \nkeele university.       | phd85@uk.ac.keele.seq1          \nkeele. staffs. England. |                            

In [191]:
# Task 1: use model_selection.train_test_split function to split your data X and Y into train and test
# Your code
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(
    X,                  # 特征数据（输入数据）
    Y,                  # 标签数据（目标数据）
    test_size=None,     # 测试集比例（默认 0.25，即 25%）
    train_size=None,    # 训练集比例（默认 None，即剩余部分）
    random_state=None,  # 随机种子（用于复现结果）
    shuffle=True,       # 是否打乱数据（默认 True）
    stratify=None       # 是否按标签分层划分（默认 None）
)

In [199]:
# Task 2: build a vocabulary of words (you may remove stop words and do some preprocessing)
# for example, if the frequency of some words less than <= 5, then just ignore them.
# so that you can have a lower dimension for your feature vectors.

from collections import Counter
from collections import defaultdict
# Your code
cutoff_freq = 5
nlp = spacy.load('en_core_web_sm')#导入预训练了的小模型，它有一些api可以帮助我们完成nlp中的操作


class textProcessor:#多使用类进行封装
    def __init__(self):
        self.word_counts = defaultdict(int)
        self.vocab = set()
        self.vocab_list=[]#排序之后的词表
        self.vocab_idx={}#word->index的映射
    def trans(self,texts):
        # if n==1:return texts[1]这个方法是错误的
        # 因为，如果是单个tuple，那么和tuple列表不一样，比如（2，“halo”）这个tuple长度是2
        if isinstance(texts,tuple):
            return [texts[1]]#一定要[text[i]]而不是text[i]，整个trans方法我们最外层包的应该是list
        n=len(texts)
        re=["0"]*n
        for i in range(n):
            re[i]=texts[i][1]
        return re
    
    def buildVocabulary(self,texts):#用所有的文本去构建出全局词表
        self.word_counts.clear()#所有 textProcessor 实例共享同一个word_counts，多次调用 buildVocabulary 会导致词频累加，因此每次我们需要清空
        texts=self.trans(texts)
        for text in texts:
            doc=nlp(text)
            for token in doc:
                if (not token.is_stop 
                    and not token.is_punct 
                    and not token.is_digit 
                    and not token.is_space 
                    and not token.is_currency
                    and token.text != '<'
                    and token.text !='>'):#用token.text把token对象转化为string对象
                    lemma = token.lemma_.lower()
                    self.word_counts[lemma] += 1
                    
        self.vocab={word for word, count in self.word_counts.items() if count > cutoff_freq}
        self.vocab_list=sorted(self.vocab)#因为我们知道只会调用一次buildVocabulary，那么把vocab_list放在这里
        for idx,word in enumerate(self.vocab_list):#语法：返回值是index-value的顺序，一定要注意
            self.vocab_idx[word]=idx
        return self.vocab
                
    def preprocess(self,text):#将原始文档预处理，转换为基于词表的特征表示
        # 词表 vocab = {'apple', 'banana', 'fruit'}
        # 原始文档 text = "I ate an apple and a Banana."
        # 预处理后 processed_text = ['apple', 'banana']  # 只保留词表中的词
        # 本来写了text=self.trans(text),但实际上我们不需要做这一步，因为我们都是在getFeatures中调用preprocess的，传入的已经是字符串了
        result=[]
        doc=nlp(text)
        for token in doc:
            if (token.lemma_.lower() in self.vocab):
                result.append(token.lemma_.lower())
        return result

    def getFeatures(self,texts):#我们此处使用最原始的方法去构建传入的texts的特征矩阵
        texts=self.trans(texts)
        re1=[]
        for text in texts:
            re1.append(self.preprocess(text))
        #self.vocab_list=sorted(self.vocab)  虽然执行上没问题，但是多次调用getFeatures就会重复初始化self.vocab_list
        mat=np.zeros((len(texts),len(self.vocab_list)))
        for i,words in enumerate(re1):
            count_dict=Counter(words)#直接使用Counter进行计数
            for word,num in count_dict.items():#语法问题，要写.items,而不是直接写count_dict
                if word in self.vocab:  # 只处理词表中的词,注意不能写vocab_list,而是用vocab这个set，不让时间复杂度高很多
                    idx=self.vocab_idx[word]
                    mat[i][idx]=num
        return mat


        
# 初始化处理器和词表
processor = textProcessor()
vocab=processor.buildVocabulary(X_train)


In [200]:
# Task 3: to represent training and testing data as word vector counts

X_train_dataset = processor.getFeatures(X_train)
X_test_dataset = processor.getFeatures(X_test)

In [201]:
# Task 4: using sklearn's Multinomial Naive Bayes class: MultinomialNB to do classification
# you may use predict() and score function to test your model on testing datasets.
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# 1. 初始化 MultinomialNB 模型
clf = MultinomialNB()

# 2. 使用训练数据训练模型
clf.fit(X_train_dataset, Y_train)

# 3. 使用训练好的模型对测试数据进行预测
Y_test_pred = clf.predict(X_test_dataset)

# 4. 评估模型性能
# 计算模型在训练集和测试集上的准确率
sklearn_score_train = clf.score(X_train_dataset, Y_train)
sklearn_score_test = clf.score(X_test_dataset, Y_test)

# 打印分类报告（包括精确率、召回率、F1分数等）
print("Classification Report for Test Data:")
print(classification_report(Y_test, Y_test_pred))

# 打印模型在训练集和测试集上的准确率
print(f"Model Accuracy on Training Data: {sklearn_score_train:.4f}")
print(f"Model Accuracy on Testing Data: {sklearn_score_test:.4f}")

Classification Report for Test Data:
                          precision    recall  f1-score   support

             alt.atheism       0.76      0.85      0.80        26
           comp.graphics       0.74      0.74      0.74        23
 comp.os.ms-windows.misc       0.72      0.78      0.75        23
comp.sys.ibm.pc.hardware       0.69      0.80      0.74        25
   comp.sys.mac.hardware       0.80      0.71      0.75        28
          comp.windows.x       0.76      0.80      0.78        20
            misc.forsale       0.90      0.86      0.88        22
               rec.autos       0.90      0.87      0.88        30
         rec.motorcycles       0.78      0.86      0.82        21
      rec.sport.baseball       0.96      0.87      0.92        31
        rec.sport.hockey       0.89      1.00      0.94        24
               sci.crypt       0.91      0.91      0.91        22
         sci.electronics       0.92      0.48      0.63        23
                 sci.med       0.95   

## 2. Logistic Regression classifcation for IMDB datasets

In [1]:
import re
import pickle
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
df = pd.read_csv('/home/leoyang/Desktop/data/data-lec3/datasets/IMDB-Dataset.csv')
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
def preprocessor(text):#此处是用正则表达式处理我们的review
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

df['review'] = df['review'].apply(preprocessor)#pandas的df对象内置了apply方法，可以对其调用自定义函数
df.head(5)

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming tech...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there s a family where a little boy ...,negative
4,petter mattei s love in the time of money is a...,positive


In [ ]:
# prepare some training and testing samples

num_samples = len(df['review'])
num_train = int(num_samples * 0.7)
num_test = int(num_samples * 0.3)

#np.random.permutation(num_samples) 会生成一个从 0 到 num_samples - 1 的随机排列的数组
#从数据集中随机选择样本时，可以使用这种方法生成随机索引。
random_indices = np.random.permutation(num_samples)

print(num_samples)

50000


In [9]:
# Task 1: prepare traing and testing datasets using pandas


# Your code
train_indices = random_indices[:num_train]
test_indices = random_indices[num_train:]
X_train = df["review"].iloc[train_indices]#iloc是基于位置的提取
y_train = df["sentiment"].iloc[train_indices]
X_test = df["review"].iloc[test_indices]
y_test = df["sentiment"].iloc[test_indices]

In [10]:
print("X_train : ",X_train.shape," X_test : ",X_test.shape,"\ny_train : ",y_train.shape," y_test : ",y_test.shape)

X_train :  (35000,)  X_test :  (15000,) 
y_train :  (35000,)  y_test :  (15000,)


In [14]:
# Task 2: Use TF-IDF matrix to as input features


# Your code be something like


#将文本数据转换为数值特征向量
#将文本数据（如 X_train）转换为一个稀疏矩阵，每一行代表一个文档，每一列代表一个词的特征值。
tfidf = TfidfVectorizer()


#作用：用于分类任务，预测目标标签。
#一开始训练发出的警告：STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.
#这个警告表示逻辑回归模型在训练时未收敛，可能是因为 max_iter（最大迭代次数）设置得太小。
classifier = LogisticRegressionCV(max_iter = 100)

# 将多个步骤（如特征提取和分类）组合成一个整体，简化代码并避免数据泄露。
# Pipeline 的参数 steps 是一个列表，列表中的每个元素是一个元组 (step_name, step_object)。
# step_name 是步骤的名称，可以自定义。
# step_object 是该步骤的对象，需要提前初始化。
my_model = Pipeline([('text_to_features',tfidf), ('my_classifier',classifier)])

my_model.fit(X_train, y_train)#开始训练模型

Pipeline(steps=[('text_to_features', TfidfVectorizer()),
                ('my_classifier', LogisticRegressionCV())])

In [15]:
# Task 3: make prediction on testing datasets


#传统的机器学习的模型采用fit和predict的方式
y_pred = my_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(print(classification_report(y_test, y_pred)))
print(accuracy_score(y_test, y_pred))

[[6676  819]
 [ 665 6840]]
              precision    recall  f1-score   support

    negative       0.91      0.89      0.90      7495
    positive       0.89      0.91      0.90      7505

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000

None
0.9010666666666667
